In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re
import lightgbm as lgb
import datetime as dt
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [2]:
borrar = ["Opportunity_Name", "ID", "Brand", "Product_Type", "ASP_converted_Currency", 
          "Prod_Category_A", "Product_Category_B", "Actual_Delivery_Date", "Last_Activity", 
          "ASP_converted_Currency", "Prod_Category_A", "Product_Name", "Delivery_Year", "Month", 
          "TRF", "Product_Family", "Account_Name"]

useless = ["Submitted_for_Approval", "Account_Type", "Delivery_Terms", "Size", "Price", "ASP_Currency", 
           "Total_Amount_Currency", "Total_Taxable_Amount_Currency","Quote_Type", "Opportunity_Type"] # "Product_Category_B", "Region"]

dates = ["Account_Created_Date", "Opportunity_Created_Date", "Quote_Expiry_Date", "Last_Modified_Date", 
         "Planned_Delivery_Start_Date", "Planned_Delivery_End_Date"]

target = ["Opportunity_ID", "Stage", "Sales_Contract_No"]

In [3]:
def preprocess(data):
    
    # elimino caracteres prohibidos en los headers
    
    data = data.rename(columns = lambda x:re.sub("[^A-Za-z0-9_]+", "", x))
        
    for d in dates:
        data[d] = pd.to_datetime(data[d])

    # agrego features
    
    data["Contacts"] = data.groupby("Opportunity_ID", sort = False)["Opportunity_ID"].transform("count")
    data["Delivery_Difference"] = (data["Planned_Delivery_End_Date"] - data["Planned_Delivery_Start_Date"]).dt.days
    data["Same_Owner"] = (data.Opportunity_Owner == data.Account_Owner) & (data.Opportunity_Owner == data.Last_Modified_By)
    data["Has_Brand"] = data.Brand != "None"
    data["Has_Contract"] = data.Sales_Contract_No != "None"
    data["Different_Country"] = (data.Billing_Country != data.Territory) & (data.Territory != "None")    
    data.loc[data.TRF == 0, "TRF_Cat"] = 0
    data.loc[(1 <= data.TRF) & (data.TRF <= 7), "TRF_Cat"] = 1
    data.loc[data.TRF > 7, "TRF_Cat"] = 2
    data["Sales"] = data.groupby("Account_Name", sort = False)["Account_Name"].transform("count")
    
    data["Concrete_Offer"] = (data["Planned_Delivery_End_Date"] - data["Opportunity_Created_Date"]).dt.days
    
    data["Offer_Duration"] = (data["Quote_Expiry_Date"] - data["Opportunity_Created_Date"]).dt.days
    
    # fabri
    
    data["Territory_Defined"] = data.Territory != "None"
    data["Past_Quote"] = (data["Last_Modified_Date"] - data["Quote_Expiry_Date"]).dt.days
    
    # casteo a categoricas varias columnas
    categorical = [x for x in data.columns if data[x].dtype == "object"]
    for c in categorical:       
        data[c] = data[c].astype('category')
        
    # limpio columnas
    
    data = data.drop(borrar + useless + dates, axis = 1)
    
    #data.drop(data[(data.Contacts > 1) & (data.ASP.isna())].index, inplace = True)
    #data = data.drop_duplicates([x for x in data.columns if x != 'ID'])
    
    return (data)

In [4]:
import catboost as ctb

In [5]:
data = pd.read_csv("Entrenamieto_ECI_2020.csv")

In [6]:
rest = [x for x in data.columns if x != 'ID']
len(data[data.duplicated(rest)])/float(len(data)) * 100

2.643535729037588

In [7]:
prep = preprocess(data)

In [8]:
prep = prep[(prep.Stage == "Closed Won") | (prep.Stage == "Closed Lost")]
prep.Stage = prep.Stage.replace({"Closed Won": 1, "Closed Lost": 0})

In [9]:
len(prep[prep.duplicated()])

481

In [10]:
x_train, x_test, y_train, y_test = train_test_split(prep, prep.Stage, test_size=0.3, random_state=0)

In [11]:
cat_vars = [x for x in x_train.drop(target, axis=1).select_dtypes('category').columns if x != 'Stage']

In [12]:
cat_vars

['Region',
 'Territory',
 'Bureaucratic_Code',
 'Source',
 'Billing_Country',
 'Account_Owner',
 'Opportunity_Owner',
 'Currency',
 'Last_Modified_By',
 'Delivery_Quarter']

In [13]:
cat_indexes = [x_train.drop(target, axis=1).columns.get_loc(x) for x in cat_vars]

In [14]:
cat_indexes

[0, 1, 6, 7, 8, 9, 10, 11, 12, 15]

In [15]:
model = ctb.CatBoostClassifier(num_boost_round=5000,
                               learning_rate=0.015,
                               l2_leaf_reg=3.5,
                               depth=8, 
                               max_bin=400,
                               num_leaves=31,
                               rsm=0.98,
                               eval_metric='Logloss',
                               use_best_model=True,
                               random_seed=42)

In [16]:
model.fit(x_train.drop(target, axis=1), y_train, eval_set=(x_test.drop(target, axis=1), y_test), cat_features=cat_indexes, early_stopping_rounds=800)

0:	learn: 0.6662320	test: 0.6666361	best: 0.6666361 (0)	total: 74.1ms	remaining: 6m 10s
1:	learn: 0.6429421	test: 0.6436226	best: 0.6436226 (1)	total: 91.7ms	remaining: 3m 49s
2:	learn: 0.6216509	test: 0.6225760	best: 0.6225760 (2)	total: 111ms	remaining: 3m 4s
3:	learn: 0.5973714	test: 0.5985329	best: 0.5985329 (3)	total: 130ms	remaining: 2m 41s
4:	learn: 0.5773562	test: 0.5786428	best: 0.5786428 (4)	total: 147ms	remaining: 2m 26s
5:	learn: 0.5590138	test: 0.5604702	best: 0.5604702 (5)	total: 155ms	remaining: 2m 8s
6:	learn: 0.5404298	test: 0.5413506	best: 0.5413506 (6)	total: 172ms	remaining: 2m 3s
7:	learn: 0.5231761	test: 0.5243156	best: 0.5243156 (7)	total: 181ms	remaining: 1m 53s
8:	learn: 0.5056932	test: 0.5075932	best: 0.5075932 (8)	total: 199ms	remaining: 1m 50s
9:	learn: 0.4882563	test: 0.4904856	best: 0.4904856 (9)	total: 215ms	remaining: 1m 47s
10:	learn: 0.4727450	test: 0.4752240	best: 0.4752240 (10)	total: 225ms	remaining: 1m 42s
11:	learn: 0.4590669	test: 0.4617270	best:

97:	learn: 0.1195648	test: 0.1300276	best: 0.1300276 (97)	total: 1.78s	remaining: 1m 29s
98:	learn: 0.1187170	test: 0.1292862	best: 0.1292862 (98)	total: 1.81s	remaining: 1m 29s
99:	learn: 0.1184590	test: 0.1290435	best: 0.1290435 (99)	total: 1.82s	remaining: 1m 29s
100:	learn: 0.1176779	test: 0.1283222	best: 0.1283222 (100)	total: 1.84s	remaining: 1m 29s
101:	learn: 0.1169994	test: 0.1277097	best: 0.1277097 (101)	total: 1.86s	remaining: 1m 29s
102:	learn: 0.1163760	test: 0.1271325	best: 0.1271325 (102)	total: 1.88s	remaining: 1m 29s
103:	learn: 0.1157885	test: 0.1266074	best: 0.1266074 (103)	total: 1.9s	remaining: 1m 29s
104:	learn: 0.1153557	test: 0.1262293	best: 0.1262293 (104)	total: 1.92s	remaining: 1m 29s
105:	learn: 0.1148145	test: 0.1257109	best: 0.1257109 (105)	total: 1.94s	remaining: 1m 29s
106:	learn: 0.1146219	test: 0.1255326	best: 0.1255326 (106)	total: 1.96s	remaining: 1m 29s
107:	learn: 0.1142098	test: 0.1251619	best: 0.1251619 (107)	total: 1.98s	remaining: 1m 29s
108:	l

192:	learn: 0.0883016	test: 0.1030906	best: 0.1030906 (192)	total: 3.68s	remaining: 1m 31s
193:	learn: 0.0880725	test: 0.1029463	best: 0.1029463 (193)	total: 3.71s	remaining: 1m 31s
194:	learn: 0.0879826	test: 0.1028495	best: 0.1028495 (194)	total: 3.73s	remaining: 1m 31s
195:	learn: 0.0877902	test: 0.1027403	best: 0.1027403 (195)	total: 3.75s	remaining: 1m 31s
196:	learn: 0.0875712	test: 0.1025750	best: 0.1025750 (196)	total: 3.77s	remaining: 1m 31s
197:	learn: 0.0874899	test: 0.1025216	best: 0.1025216 (197)	total: 3.79s	remaining: 1m 31s
198:	learn: 0.0873775	test: 0.1024305	best: 0.1024305 (198)	total: 3.81s	remaining: 1m 31s
199:	learn: 0.0872168	test: 0.1022754	best: 0.1022754 (199)	total: 3.83s	remaining: 1m 31s
200:	learn: 0.0869667	test: 0.1020318	best: 0.1020318 (200)	total: 3.85s	remaining: 1m 31s
201:	learn: 0.0868637	test: 0.1019846	best: 0.1019846 (201)	total: 3.88s	remaining: 1m 32s
202:	learn: 0.0868418	test: 0.1019492	best: 0.1019492 (202)	total: 3.9s	remaining: 1m 32s


283:	learn: 0.0754156	test: 0.0934579	best: 0.0934579 (283)	total: 6.01s	remaining: 1m 39s
284:	learn: 0.0753467	test: 0.0934132	best: 0.0934132 (284)	total: 6.04s	remaining: 1m 39s
285:	learn: 0.0751822	test: 0.0932636	best: 0.0932636 (285)	total: 6.06s	remaining: 1m 39s
286:	learn: 0.0750065	test: 0.0932067	best: 0.0932067 (286)	total: 6.08s	remaining: 1m 39s
287:	learn: 0.0749518	test: 0.0931440	best: 0.0931440 (287)	total: 6.11s	remaining: 1m 39s
288:	learn: 0.0748420	test: 0.0930404	best: 0.0930404 (288)	total: 6.13s	remaining: 1m 39s
289:	learn: 0.0748037	test: 0.0930220	best: 0.0930220 (289)	total: 6.16s	remaining: 1m 39s
290:	learn: 0.0747661	test: 0.0930077	best: 0.0930077 (290)	total: 6.18s	remaining: 1m 39s
291:	learn: 0.0746975	test: 0.0929629	best: 0.0929629 (291)	total: 6.2s	remaining: 1m 39s
292:	learn: 0.0746171	test: 0.0929226	best: 0.0929226 (292)	total: 6.22s	remaining: 1m 39s
293:	learn: 0.0744937	test: 0.0928537	best: 0.0928537 (293)	total: 6.25s	remaining: 1m 40s


376:	learn: 0.0673179	test: 0.0879425	best: 0.0879425 (376)	total: 7.94s	remaining: 1m 37s
377:	learn: 0.0672593	test: 0.0879204	best: 0.0879204 (377)	total: 7.96s	remaining: 1m 37s
378:	learn: 0.0671375	test: 0.0878782	best: 0.0878782 (378)	total: 7.98s	remaining: 1m 37s
379:	learn: 0.0670964	test: 0.0878392	best: 0.0878392 (379)	total: 8s	remaining: 1m 37s
380:	learn: 0.0670371	test: 0.0878069	best: 0.0878069 (380)	total: 8.02s	remaining: 1m 37s
381:	learn: 0.0669159	test: 0.0877223	best: 0.0877223 (381)	total: 8.04s	remaining: 1m 37s
382:	learn: 0.0668881	test: 0.0877116	best: 0.0877116 (382)	total: 8.05s	remaining: 1m 37s
383:	learn: 0.0667998	test: 0.0875992	best: 0.0875992 (383)	total: 8.07s	remaining: 1m 37s
384:	learn: 0.0667881	test: 0.0875985	best: 0.0875985 (384)	total: 8.09s	remaining: 1m 37s
385:	learn: 0.0667763	test: 0.0875928	best: 0.0875928 (385)	total: 8.11s	remaining: 1m 36s
386:	learn: 0.0666996	test: 0.0875457	best: 0.0875457 (386)	total: 8.13s	remaining: 1m 36s
38

...

4125:	learn: 0.0112912	test: 0.0614968	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.4s
4126:	learn: 0.0112908	test: 0.0615031	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.4s
4127:	learn: 0.0112883	test: 0.0615136	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.4s
4128:	learn: 0.0112883	test: 0.0615136	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.3s
4129:	learn: 0.0112883	test: 0.0615136	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.3s
4130:	learn: 0.0112872	test: 0.0615155	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.3s
4131:	learn: 0.0112850	test: 0.0615243	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.3s
4132:	learn: 0.0112798	test: 0.0615295	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.3s
4133:	learn: 0.0112798	test: 0.0615294	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.2s
4134:	learn: 0.0112798	test: 0.0615294	best: 0.0613762 (3713)	total: 1m 31s	remaining: 19.2s
4135:	learn: 0.0112798	test: 0.0615294	best: 0.0613762 (3713)	total: 1

4224:	learn: 0.0111626	test: 0.0614966	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17.2s
4225:	learn: 0.0111626	test: 0.0614966	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17.1s
4226:	learn: 0.0111626	test: 0.0614966	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17.1s
4227:	learn: 0.0111626	test: 0.0614966	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17.1s
4228:	learn: 0.0111626	test: 0.0614966	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17.1s
4229:	learn: 0.0111462	test: 0.0614913	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17s
4230:	learn: 0.0111462	test: 0.0614915	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17s
4231:	learn: 0.0111462	test: 0.0614915	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17s
4232:	learn: 0.0111461	test: 0.0614925	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17s
4233:	learn: 0.0111461	test: 0.0614925	best: 0.0613762 (3713)	total: 1m 33s	remaining: 17s
4234:	learn: 0.0111461	test: 0.0614925	best: 0.0613762 (3713)	total: 1m 33s	rema

4315:	learn: 0.0110421	test: 0.0615060	best: 0.0613762 (3713)	total: 1m 35s	remaining: 15.1s
4316:	learn: 0.0110421	test: 0.0615060	best: 0.0613762 (3713)	total: 1m 35s	remaining: 15.1s
4317:	learn: 0.0110328	test: 0.0615147	best: 0.0613762 (3713)	total: 1m 35s	remaining: 15s
4318:	learn: 0.0110317	test: 0.0615127	best: 0.0613762 (3713)	total: 1m 35s	remaining: 15s
4319:	learn: 0.0110317	test: 0.0615126	best: 0.0613762 (3713)	total: 1m 35s	remaining: 15s
4320:	learn: 0.0110317	test: 0.0615129	best: 0.0613762 (3713)	total: 1m 35s	remaining: 15s
4321:	learn: 0.0110317	test: 0.0615129	best: 0.0613762 (3713)	total: 1m 35s	remaining: 14.9s
4322:	learn: 0.0110317	test: 0.0615130	best: 0.0613762 (3713)	total: 1m 35s	remaining: 14.9s
4323:	learn: 0.0110317	test: 0.0615130	best: 0.0613762 (3713)	total: 1m 35s	remaining: 14.9s
4324:	learn: 0.0110244	test: 0.0615132	best: 0.0613762 (3713)	total: 1m 35s	remaining: 14.9s
4325:	learn: 0.0110196	test: 0.0615089	best: 0.0613762 (3713)	total: 1m 35s	re

4407:	learn: 0.0109313	test: 0.0615400	best: 0.0613762 (3713)	total: 1m 36s	remaining: 13s
4408:	learn: 0.0109199	test: 0.0615241	best: 0.0613762 (3713)	total: 1m 36s	remaining: 13s
4409:	learn: 0.0109159	test: 0.0615291	best: 0.0613762 (3713)	total: 1m 36s	remaining: 13s
4410:	learn: 0.0109159	test: 0.0615291	best: 0.0613762 (3713)	total: 1m 36s	remaining: 12.9s
4411:	learn: 0.0109159	test: 0.0615291	best: 0.0613762 (3713)	total: 1m 36s	remaining: 12.9s
4412:	learn: 0.0109159	test: 0.0615290	best: 0.0613762 (3713)	total: 1m 36s	remaining: 12.9s
4413:	learn: 0.0109159	test: 0.0615291	best: 0.0613762 (3713)	total: 1m 36s	remaining: 12.9s
4414:	learn: 0.0109159	test: 0.0615291	best: 0.0613762 (3713)	total: 1m 36s	remaining: 12.8s
4415:	learn: 0.0109159	test: 0.0615291	best: 0.0613762 (3713)	total: 1m 36s	remaining: 12.8s
4416:	learn: 0.0109146	test: 0.0615323	best: 0.0613762 (3713)	total: 1m 37s	remaining: 12.8s
4417:	learn: 0.0109146	test: 0.0615323	best: 0.0613762 (3713)	total: 1m 37s	

4496:	learn: 0.0107617	test: 0.0615433	best: 0.0613762 (3713)	total: 1m 38s	remaining: 11s
4497:	learn: 0.0107616	test: 0.0615432	best: 0.0613762 (3713)	total: 1m 38s	remaining: 11s
4498:	learn: 0.0107616	test: 0.0615432	best: 0.0613762 (3713)	total: 1m 38s	remaining: 11s
4499:	learn: 0.0107612	test: 0.0615496	best: 0.0613762 (3713)	total: 1m 38s	remaining: 11s
4500:	learn: 0.0107505	test: 0.0615318	best: 0.0613762 (3713)	total: 1m 38s	remaining: 10.9s
4501:	learn: 0.0107505	test: 0.0615318	best: 0.0613762 (3713)	total: 1m 38s	remaining: 10.9s
4502:	learn: 0.0107505	test: 0.0615318	best: 0.0613762 (3713)	total: 1m 38s	remaining: 10.9s
4503:	learn: 0.0107505	test: 0.0615318	best: 0.0613762 (3713)	total: 1m 38s	remaining: 10.9s
4504:	learn: 0.0107504	test: 0.0615321	best: 0.0613762 (3713)	total: 1m 38s	remaining: 10.8s
4505:	learn: 0.0107429	test: 0.0615252	best: 0.0613762 (3713)	total: 1m 38s	remaining: 10.8s
4506:	learn: 0.0107429	test: 0.0615252	best: 0.0613762 (3713)	total: 1m 38s	re

In [17]:
y_pred = model.predict_proba(x_test.drop(target, axis=1))

In [18]:
res = y_pred[:,1]

In [19]:
for x in res[:10]:
    print("{:f}".format(x))

0.000184
0.001800
0.998842
0.994814
0.315044
0.965229
0.006447
0.000359
0.998852
0.000518


In [20]:
x_test.Stage.head(10)

2040     0
4069     0
4230     1
578      1
12595    0
6042     1
2763     0
15932    0
2669     1
12803    0
Name: Stage, dtype: int64

In [21]:
fuck = [x for x in res if x >= 0.025 and x <= 0.95]
len(fuck)

350

In [22]:
len(fuck)/float(len(res)) * 100

6.910167818361303

In [23]:
pred = pd.DataFrame(res, index = x_test.index, columns = ["Prediction"])

In [24]:
x_test_pred = x_test.join(pred)
answer = x_test_pred.groupby("Opportunity_ID")["Stage","Prediction"].mean()

In [25]:
log_loss(answer["Stage"], answer["Prediction"])

0.0704374218742913

In [26]:
predictors = [x for x in x_train.drop(target, axis = 1).columns]
feat_imp = pd.Series(model.feature_importances_, predictors).sort_values(ascending = False)
feat_imp = feat_imp[0:28]
plt.rcParams['figure.figsize'] = 28, 5
feat_imp.plot(kind = 'bar', title = 'Feature Importance')

In [27]:
bad = x_test_pred[(x_test_pred.Prediction > 0.5) & (x_test_pred.Stage == 0) | (x_test_pred.Prediction < 0.5) & (x_test_pred.Stage == 1)]

In [28]:
bad

,Region,Territory,PricingDelivery_Terms_Quote_Appr,PricingDelivery_Terms_Approved,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Source,Billing_Country,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Currency,Last_Modified_By,ASP,ASP_converted,Delivery_Quarter,Total_Amount,Total_Taxable_Amount,Stage,Contacts,Delivery_Difference,Same_Owner,Has_Brand,Has_Contract,Different_Country,TRF_Cat,Sales,Concrete_Offer,Offer_Duration,Territory_Defined,Past_Quote,Prediction
7812,APAC,Australia,1,0,1,0,Bureaucratic_Code_5,None,Australia,4832,2252,Person_Name_43,Person_Name_19,None,Person_Name_47,0.5400,0.38211,Q4,109620.00,109620.00,0,1,7.0,False,False,True,False,0.0,317,181.0,130.0,True,131.0,0.969321
4773,APAC,India,1,1,1,1,Bureaucratic_Code_4,Source_13,Sri Lanka,2824,904,Person_Name_49,Person_Name_49,None,Person_Name_47,0.3600,0.36000,Q4,157172.40,157172.40,0,1,19.0,False,False,True,True,0.0,31,72.0,72.0,True,-27.0,0.996730
9545,EMEA,Germany,0,0,0,0,Bureaucratic_Code_4,Source_9,Germany,6045,None,Person_Name_13,Person_Name_13,None,Person_Name_47,0.4200,0.47506,Q3,30450.00,82700.00,1,2,4.0,False,False,False,False,0.0,125,32.0,4.0,True,160.0,0.365609
9800,APAC,Thailand,0,0,0,0,Bureaucratic_Code_4,Source_7,Thailand,6207,3436,Person_Name_54,Person_Name_54,None,Person_Name_47,0.4000,0.40000,Q4,120060.00,120060.00,0,1,0.0,False,False,True,False,0.0,1,86.0,57.0,True,70.0,0.977528
952,EMEA,Germany,1,0,0,0,Bureaucratic_Code_5,Source_7,Germany,511,1344,Person_Name_4,Person_Name_4,None,Person_Name_41,0.3875,0.43830,Q2,287525.00,287525.00,0,1,12.0,False,False,True,False,1.0,17,88.0,8.0,True,603.0,0.993678
16800,EMEA,None,0,0,0,0,Bureaucratic_Code_4,None,Luxembourg,12726,None,Person_Name_18,Person_Name_20,None,Person_Name_20,0.5400,0.61079,Q1,105300.00,105300.00,1,1,3.0,False,False,False,False,0.0,2,86.0,72.0,False,-8.0,0.113047
9280,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,None,United States,5834,2880,Person_Name_64,Person_Name_64,None,Person_Name_47,0.4300,0.43000,Q4,30702.00,30702.00,0,1,0.0,False,False,True,True,0.0,22,132.0,4.0,True,183.0,0.630035
10407,Americas,None,1,0,0,0,Bureaucratic_Code_5,Source_7,United States,6515,3390,Person_Name_33,Person_Name_33,None,Person_Name_47,0.4950,0.49500,Q1,285862.50,285862.50,0,1,30.0,False,False,True,False,1.0,73,89.0,NaN,False,NaN,0.978764
15533,EMEA,Germany,1,1,1,1,Bureaucratic_Code_4,Source_7,Germany,9934,None,Person_Name_62,Person_Name_62,None,Person_Name_62,0.2300,0.26015,Q4,10666.25,10666.25,1,1,2.0,True,False,False,False,0.0,39,6.0,NaN,True,NaN,0.180077
7806,APAC,Australia,1,1,1,1,Bureaucratic_Code_4,None,Australia,4826,2247,Person_Name_43,Person_Name_19,None,Person_Name_47,0.5400,0.38211,Q4,109620.00,109620.00,0,1,7.0,False,False,True,False,0.0,317,181.0,130.0,True,131.0,0.813893


In [29]:
bad.Stage.value_counts()

0    78
1    21
Name: Stage, dtype: int64

In [30]:
validation_file = "Validacion_ECI_2020.csv"
vali = pd.read_csv(validation_file)
validation = preprocess(vali)
leak = ["Opportunity_ID", "Sales_Contract_No"]
pred = model.predict_proba(validation.drop(leak, axis = 1))[:,1]

# agrupo por Opportunity_ID para dar una sola prediccion por solicitud

pred = pd.DataFrame(pred, index = validation.index, columns = ["Prediction"])
validation = validation.join(pred)

answer = pd.DataFrame(validation.groupby("Opportunity_ID", as_index = False)["Prediction"].mean())

In [31]:
prev = pd.read_csv("acceptable/submission_23.csv", names=["Opportunity_ID", "Prediction"])
prev["Prediction"].corr(answer["Prediction"])

0.9966014072053296

In [32]:
prev = pd.read_csv("acceptable/submission_11.csv")
prev["Prediction"].corr(answer["Prediction"])

0.993107873090827

In [37]:
answer.to_csv("acceptable/submission_26.csv", header = False, index = False)

In [34]:
#p = prev.merge(answer, on = "Opportunity_ID")

In [35]:
#p["New_Prediction"] = (p.Prediction_x + p.Prediction_y) / 2.0

In [36]:
#pp = p[["Opportunity_ID", "New_Prediction"]]
#pp.to_csv("submission_24.csv", header = False, index = False)